## مسابقه بزرگ هوش مصنوعی

In [ ]:
#Ali Badrloo
#Mahdi Alikhani
#Mohammadjavad Ghazikhani

In [10]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model

In [11]:
# Define your folder structure
data_dir = 'trimmed_audio'
classes = ['ali_ghaderi', 'amin_taheri', 'faezeh_najafi', 'houman', 'kourosh',
            'mahdi', 'mahdi_joozdani', 'mani_hosseini', 'mehdi_gozali', 'mojtaba',
              'nazanin_hasani', 'negar', 'saba', 'sam', 'samyar_miri']

In [ ]:
# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []
    
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):

                file_path = os.path.join(class_dir, filename)

                #Pre-Processing , trimming(scilence and noise)
                audio_data, sample_rate = librosa.load(file_path, sr=None)

                # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                
                data.append(mel_spectrogram)
                labels.append(i)
    
    return np.array(data), np.array(labels)

In [13]:
# Split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

c:\Users\A\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1631
  warnings.warn(


In [14]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_train:", y_train.shape)
print("Shape of Y_test:", y_test.shape)

Shape of X_train: (245, 128, 128, 1)
Shape of X_test: (62, 128, 128, 1)
Shape of Y_train: (245, 15)
Shape of Y_test: (62, 15)


In [15]:
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# Define the feature extractor (using the convolutional layers)
input_shape = X_train[0].shape  # Example: (128, 128, 3)
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
feature_extractor = Model(inputs=input_layer, outputs=x)

# Extract features from training data
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

print("Extracted features (train):", X_train_features.shape)  # Example: (num_samples, num_features)
print("Extracted features (test):", X_test_features.shape)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Extracted features (train): (245, 57600)
Extracted features (test): (62, 57600)


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Convert one-hot encoded labels to class indices
y_train_indices = np.argmax(y_train, axis=1)
y_test_indices = np.argmax(y_test, axis=1)

# Train XGBoost
xgb_model = XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_features, y_train_indices)

# Predict and evaluate
y_pred = xgb_model.predict(X_test_features)
accuracy = accuracy_score(y_test_indices, y_pred)
print("XGBoost Test Accuracy:", accuracy)

c:\Users\A\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:49:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Test Accuracy: 0.7741935483870968


In [ ]:
# Save the XGBoost model
xgb_model.save_model('xgb_model.pkl')

c:\Users\A\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:03:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
